# 类与对象

## 改变对象的字符串显示
问题：想改变对象实例的打印内容，增加可读性
解决：重新定义 __ str __ () 和 __ repr __ () 方法

- 使用 repr() 函数返回 __ repr __ () 方法定义的字符串
- 使用 str() 或 print() 函数会返回 __ str __ () 方法定义的字符串

In [5]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'MyPair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)

p = Pair(3, 4)
print(repr(p))
print(p)

MyPair(3, 4)
(3, 4)


## 自定义字符串的格式化
问题：想通过 format() 函数和字符串方法使得一个对象能支持自定义的格式化
解决：在类上面定义 format () 方法

In [7]:
_formats = { 'ymd' : '{d.year}-{d.month}-{d.day}', 
             'mdy' : '{d.month}/{d.day}/{d.year}', 
             'dmy' : '{d.day}/{d.month}/{d.year}' }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [9]:
d = Date(2012, 12, 21)
print(format(d))
print('The date is {:ymd}'.format(d))
print('The date is {:mdy}'.format(d))

2012-12-21
The date is 2012-12-21
The date is 12/21/2012


In [10]:
from datetime import date
d = date(2012, 12, 21)
print(format(d))
print(format(d,'%A, %B %d, %Y'))

2012-12-21
Friday, December 21, 2012


## 创建大量对象时节省内存方法
问题：程序要创建大量 (可能上百万) 的对象，导致占用很大的内存，怎样压缩空间？
解答：给类添加 __ slots __ 属性来极大的减少实例所占的内存
- 优点：这样实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典
- 不能再给实例添加新的属性了，只能使用在 slots 中定义的那些属性名
- 应用场景：需要创建某个类的几百万个实例对象

In [ ]:
class Date:
    __slots__ = ['year', 'month', 'day']
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
